#<b>Automatic License Plate Recognition and Character Extraction using YOLOv3 Object Detection Algorithm and Tesseract Optical Character Recognition Engine</b><br><br>



##<b>Exploration of Enviroment, Installation of Dependencies, and Mounting of Google Drive</b>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-850f3a54-65d0-9d15-da3f-55a0ced5397b)


In [ ]:
!nvidia-smi

Sun May 22 12:08:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !sudo apt install tesseract-ocr
# !pip install pytesseract

In [ ]:
# !pip install Pillow==9.0.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/data/archive.zip

Archive:  drive/MyDrive/data/archive.zip
  inflating: classes.names           
  inflating: darknet-yolov3.cfg      
  inflating: lapi.weights            


##<b>Import Packages</b>

In [ ]:
import os
import re
import cv2
import time
import math
import shutil
import imutils
import pytesseract
import numpy as np
from PIL import Image
import IPython.core.display as ipd

##<b>Helper Functions</b>

In [ ]:
def convert(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

In [ ]:
def preprocess_plates(image, curFrame, x, y, w, h):
  if not os.path.exists("/content/drive/MyDrive/NPR/Vehicle"):
    os.makedirs("/content/drive/MyDrive/NPR/Vehicle")
  if not os.path.exists("/content/drive/MyDrive/NPR/Plates"):
    os.makedirs("/content/drive/MyDrive/NPR/Plates")
  if not os.path.exists("/content/drive/MyDrive/NPR/Preprocessed"):
    os.makedirs("/content/drive/MyDrive/NPR/Preprocessed")
  

  name = "/content/drive/MyDrive/NPR/Vehicle/frame"+str(curFrame)+".jpg"
  name2 = "/content/drive/MyDrive/NPR/Plates/plateframe"+str(curFrame)+".jpg"
  name3 = "/content/drive/MyDrive/NPR/Preprocessed/licenseplate"+str(curFrame)+".jpg"

  cv2.imwrite(name,image)
  image=Image.open(name)
  crpimg=image.crop((x, y, x+w, y+h))
  crpimg.save(name2)

  image = cv2.imread(name2)
  final_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
  final_image = cv2.morphologyEx(final_image, cv2.MORPH_BLACKHAT, rectKern)
  clahe = cv2.createCLAHE(clipLimit = 7, tileGridSize = (1,200))
  final_image = clahe.apply(final_image)
  final_image = cv2.threshold(final_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

  cv2.imwrite(name3,final_image)

In [ ]:
final_result = []
result_videos = []

def save_results(MP4_VIDEO_PATH, number_list):
  
  result = set(number_list)
  final_result_slice = []

  for x in result:
    if not x == 'THERE ARE NO LICENSE PLATES IN THE GIVEN VIDEO':
      clean_ans = re.findall("^TN[0-9]{1,2}[A-Z]{1,2}[0-9]{4}", x)
      clean_ans_str = ''.join([str(elem) for elem in clean_ans])
      if clean_ans:
        final_result_slice.append(clean_ans_str)
    else:
      final_result_slice.append(x)
  
  final_result.append(final_result_slice)
  result_videos.append(MP4_VIDEO_PATH)

##<b>YOLOv3 - License Plate Detector</b>

In [ ]:
# ! wget "https://pjreddie.com/media/files/yolov3.weights"


In [ ]:
def detect_and_store_license_plate(VIDEO_PATH):


  weightsPath = "lapi.weights"
  configPath = "darknet-yolov3.cfg"
  labelsPath = "classes.names"
  LABELS = open(labelsPath).read().strip().split("\n")

  AVI_DIRECTORY = "/content/drive/MyDrive/NPR/Result_Videos/AVI_Format/"
  result_video_name = VIDEO_PATH.split("/")[-1]
  result_video_name = result_video_name.split(".")[-2]
  AVI_VIDEO_PATH = AVI_DIRECTORY + result_video_name + str("-Result.avi")

  net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
  cap = cv2.VideoCapture(VIDEO_PATH)

  curFrame=0
  writer = None
  (W, H) = (None, None)

  try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
      else cv2.CAP_PROP_FRAME_COUNT
    total = int(cap.get(prop))
    print("{} Total Frames in Video".format(total))

  except:
    print("Could Not Determine # of Frames in Video")
    print("No Approximate Completion Time Can Be Provided")
    total = -1

  while(cap.isOpened()):
      
      ret,image=cap.read()
      if not ret:
          break
      image=cv2.resize(image,(1280,720))
      (H, W) = image.shape[:2]
      ln = net.getLayerNames()
      ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
      blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
      net.setInput(blob)
      start = time.time()
      layerOutputs = net.forward(ln)
      end = time.time()
      print("Frame Prediction Time : {:.6f} seconds".format(end - start))
      boxes = []
      confidences = []
      classIDs = []
      for output in layerOutputs:
          for detection in output:
              scores = detection[5:]
              classID = np.argmax(scores)
              confidence = scores[classID]
              if confidence > 0.1:
                  box = detection[0:4] * np.array([W, H, W, H])
                  (centerX, centerY, width, height) = box.astype("int")
                  x = int(centerX - (width / 2))
                  y = int(centerY - (height / 2))
                  boxes.append([x, y, int(width), int(height)])
                  confidences.append(float(confidence))
                  classIDs.append(classID)
                  
      idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)
      ind = []
      for i in range(0,len(classIDs)):
          ind.append(i)

      if len(idxs) > 0:
          for i in idxs.flatten():

                  (x, y) = (boxes[i][0], boxes[i][1])
                  (w, h) = (boxes[i][2], boxes[i][3])
                  preprocess_plates(image, curFrame, x, y, w, h)
                  curFrame+=1
                  color = (0,255,0)
                  cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                  text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
                  cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)   
      
      if writer is None:
          fourcc = cv2.VideoWriter_fourcc(*"MJPG")
          writer = cv2.VideoWriter(AVI_VIDEO_PATH, fourcc, 24, (image.shape[1], image.shape[0]), True)
          if total > 0:
              elap = (end - start)
              print("Single Frame Took {:.4f} Seconds".format(elap))
              print("Estimated Total Time To Finish(HH:MM:SS): ",convert(math.ceil(elap * total)))
              
      writer.write(image)

  print("Cleaning Up...")
  print("")
  print("")
  writer.release()
  cap.release()
  cv2.destroyAllWindows()
  return AVI_VIDEO_PATH

##<b>Tessaract OCR - License Number Extractor</b>

In [ ]:
def license_number_extractor(PREPROCESSED_IMAGE_PATH, number_list):

  text = pytesseract.image_to_string(Image.open(PREPROCESSED_IMAGE_PATH))
  text = text.upper()
  cleaned_list = re.findall("\w", text)
  temp_cleaned_str = ''.join([str(elem) for elem in cleaned_list])
  number_list.append(temp_cleaned_str)

  return number_list

##<b>Digital Assignment Solution Driver Function</b>

In [ ]:
def ANPR(PATH_TO_VIDEO):

  number_list = []
  STATUS0 = 'THERE ARE NO LICENSE PLATES IN THE GIVEN VIDEO'
  AVI_VIDEO_PATH = detect_and_store_license_plate(PATH_TO_VIDEO)
  MP4_VIDEO_PATH = AVI_VIDEO_PATH
  DIRECTORY = "/content/drive/MyDrive/NPR/Preprocessed/"

  if not os.path.exists(DIRECTORY):
    number_list.append(STATUS0)

  else:
    for license_plate in os.listdir(DIRECTORY):
      if license_plate.endswith('.jpg'):
        PREPROCESSED_IMAGE_PATH = DIRECTORY + str(license_plate)
        number_list = license_number_extractor(PREPROCESSED_IMAGE_PATH, number_list)
  
  save_results(MP4_VIDEO_PATH, number_list)

  if os.path.exists(DIRECTORY):
    shutil.rmtree(DIRECTORY)

##<b>Evaluation on Videos in Test Video Folder</b>

In [ ]:
TEST_VIDEOS_PATH = "/content/"

for video in os.listdir(TEST_VIDEOS_PATH):
  if video.endswith(".mp4"):
    PATH_TO_VIDEO = TEST_VIDEOS_PATH + str(video)
    ANPR(PATH_TO_VIDEO)
    
print("")
print("---------- RESULTS GENERATED SUCESSFULLY ---------- ")

61 Total Frames in Video
Frame Prediction Time : 2.147649 seconds
Single Frame Took 2.1476 Seconds
Estimated Total Time To Finish(HH:MM:SS):  00:02:12
Frame Prediction Time : 1.962652 seconds
Frame Prediction Time : 1.988070 seconds
Frame Prediction Time : 1.952920 seconds
Frame Prediction Time : 1.999191 seconds
Frame Prediction Time : 1.963550 seconds
Frame Prediction Time : 1.958005 seconds
Frame Prediction Time : 1.925881 seconds
Frame Prediction Time : 1.964032 seconds
Frame Prediction Time : 1.953949 seconds
Frame Prediction Time : 1.957541 seconds
Frame Prediction Time : 1.950432 seconds
Frame Prediction Time : 1.958975 seconds
Frame Prediction Time : 1.972672 seconds
Frame Prediction Time : 1.977085 seconds
Frame Prediction Time : 1.970954 seconds
Frame Prediction Time : 1.978248 seconds
Frame Prediction Time : 1.956974 seconds
Frame Prediction Time : 1.965029 seconds
Frame Prediction Time : 1.980319 seconds
Frame Prediction Time : 1.949753 seconds
Frame Prediction Time : 1.975

##<b>Display Results</b>

In [ ]:
for i in range(0,len(result_videos)):
  print("RESULTS FROM VIDEO:",i+1)
  print("LICENSE PLATE NUMBERS DETECTED FROM VIDEO:",i+1)
  print(final_result[i])
  print("VIDEO (WITH DETECTIONS):",i+1)
  ipd.display(ipd.Video(result_videos[i],embed=True))

RESULTS FROM VIDEO: 1
LICENSE PLATE NUMBERS DETECTED FROM VIDEO: 1
['TN12D8087', 'TN42D8037', 'TN12D8037', 'TN12D0809', 'TN12OO8037']
VIDEO (WITH DETECTIONS): 1
